In [65]:
import numpy as np
import pandas as pd
import datetime

In [66]:
#COUNTING NEXT DAY
#DATE FOR THE FIRST DAY OF GROUP IS EQUAL TO CURRENT DATE + GROUP ID
def add_date(g):
    g['date'] = datetime.datetime.now() + pd.TimedeltaIndex(g['R'], unit = 'D')
    g = g.drop('R', 1)
    return g

In [67]:
data = pd.read_csv("traffic_report.csv")

In [68]:
data.shape

(42, 3)

In [69]:
data = data[data['blogId'] != ' ']

In [70]:
data

,blogId,views,clicks
0,1,10,3
1,1,14,5
2,2,10,3
3,3,12,8
4,4,15,10
5,5,10,8
6,1,-12,9
7,2,16,13
8,2,15,
10,4,12,6


In [71]:
#CHANGING AN OBJECT TYPE TO INT AND ADDING NAN TO MISSING VALUES
data['views'] = pd.to_numeric(data['views'], errors = "coerce", downcast = "integer")
#IN CASE OF NEGATIVE VIEW VALUES, ELIMINATION OF NEGATIVE SIGN
data['views'] = data['views'].abs()
def previous_views(g):
    #if:
    #1. doesn't exist two previous values with the same blogId as the missing value
    #2. doesn't exist two next values with the same blogId as the missing value
    #3. doesn't exist previous and next value with the same blogId as the missing value 
    #if none of these conditions is valid, row with the missing value is deleted
    g = g.drop(g[((np.isnan(g.views.shift(-1))) | (np.isnan(g.views.shift(-2)))) & ((np.isnan(g.views.shift(-1))) | (np.isnan(g.views.shift(1)))) 
                 & ((np.isnan(g.views.shift(1))) | (np.isnan(g.views.shift(2))))].index)
    #deciding which positions will be used for counting average value
    conditions = [(~np.isnan(g.views.shift(-1))) & (~np.isnan(g.views.shift(1))), 
                 (~np.isnan(g.views.shift(-1))) & (~np.isnan(g.views.shift(-2))),
                 (~np.isnan(g.views.shift(1))) & (~np.isnan(g.views.shift(2)))]
    #count average value
    choices = [( g.views.shift(-1) + g.views.shift(1)) / 2,
              (g.views.shift(-1) + g.views.shift(-2)) / 2,
              (g.views.shift(1) + g.views.shift(2)) / 2]
    g['views'] = np.select(conditions, choices, default = '')
    return g

In [72]:
#APPLYING THE SAME OPERATIONS AS ON VIEWS
data['clicks'] = pd.to_numeric(data['clicks'], errors = "coerce", downcast = "integer")
data['clicks'] = data['clicks'].abs()
def previous_clicks(g):
    g = g.drop(g[((np.isnan(g.clicks.shift(-1))) | (np.isnan(g.clicks.shift(-2)))) & ((np.isnan(g.clicks.shift(-1))) | (np.isnan(g.clicks.shift(1)))) 
                 & ((np.isnan(g.clicks.shift(1))) | (np.isnan(g.clicks.shift(2))))].index)
    conditions = [(~np.isnan(g.clicks.shift(-1))) & (~np.isnan(g.clicks.shift(1))), 
                 (~np.isnan(g.clicks.shift(-1))) & (~np.isnan(g.clicks.shift(-2))),
                 (~np.isnan(g.clicks.shift(1))) & (~np.isnan(g.clicks.shift(2)))]
    choices = [( g.clicks.shift(-1) + g.clicks.shift(1)) / 2,
              (g.clicks.shift(-1) + g.clicks.shift(-2)) / 2,
              (g.clicks.shift(1) + g.clicks.shift(2)) / 2]
    g['clicks'] = np.select(conditions, choices, default = '')
    return g

In [73]:
#APPLYING DEFINED FUNCTIONS ON EACH GROUP
data = data.groupby(data.blogId).apply(previous_views)
data = data.groupby(data.blogId).apply(previous_clicks)

In [74]:
#ADDING NEW COLUMN TO OUTPUT CSV
#FUNCTION ADD_DATE WILL ADD DATES TO EACH ENTRY OF GROUP
data['R'] = data.groupby(data.blogId).cumcount()
data = data.groupby(data.blogId).apply(add_date)

In [76]:
#OUTPUT IS WRITEN IN INTEGER FORMAT
desired_decimals = 0    
data['views'] = data['views'].apply(lambda x: round(float(x),desired_decimals))
data['clicks'] = data['clicks'].apply(lambda x: round(float(x),desired_decimals))
data['views'] = data['views'].astype(int)
data['clicks'] = data['clicks'].astype(int)
data.to_csv("output.csv", index = False)